In [1]:
pip install beautifulsoup4 selenium requests webdriver-manager


   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.4 MB 4.8 MB/s eta 0:00:02
   --------------- ------------------------ 3.7/9.4 MB 10.4 MB/s eta 0:00:01
   ----------------------- ---------------- 5.5/9.4 MB 10.5 MB/s eta 0:00:01
   --------------------------------- ------ 7.9/9.4 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  9.2/9.4 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
#C:\Users\brian\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe

In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Initialize the WebDriver using the Service object
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Step 1: Navigate to Congress.gov search page for the 118th Congress bills on financial AI in Finance and Financial Sector policy area
search_url = 'https://www.congress.gov/quick-search/legislation?wordsPhrases=Artificial+Intelligence&wordVariants=on&congressGroups%5B0%5D=0&congresses%5B0%5D=all&legislationNumbers=&legislativeAction=&sponsor=on&representative=&senator=&q=%7B%22subject%22%3A%22Finance+and+Financial+Sector%22%2C%22congress%22%3A%22all%22%7D'
driver.get(search_url)

time.sleep(5)  # Wait for the page to load

# Step 2: Parse the search results page
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract all bill links from the search results by filtering out non-legislative items
bill_links = []

# Congress.gov uses a class 'result-heading' for each bill result
for item in soup.select('li[class*="expanded"]'):
    # Get the link to the bill
    link_tag = item.select_one('.result-heading a')
    if link_tag:
        bill_links.append(link_tag['href'])

# Initialize an empty list to store bill details
bills_data = []

# Helper function to add spaces where needed
def add_spaces(text):
    # Insert spaces between lowercase-uppercase and digits-letters transitions
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', text)
    text = re.sub(r'([0-9])([a-zA-Z])', r'\1 \2', text)
    return text

# Step 3: Visit each bill link and scrape detailed information including the detailed summary
for link in bill_links:
    driver.get(f"https://www.congress.gov{link}")
    time.sleep(3)  # Allow the page to load

    # Parse the detailed bill page
    bill_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract bill title, if present
    bill_title_tag = bill_soup.find('h1', class_='legDetail')
    bill_title = bill_title_tag.get_text(strip=True) if bill_title_tag else "N/A"
    
    # Extract sponsor, if present
    sponsor_tag = bill_soup.find('a', title='Sponsor')
    sponsor = sponsor_tag.get_text(strip=True) if sponsor_tag else "N/A"
    
    # Extract status, if present
    status_tag = bill_soup.find('div', class_='overview')
    status = status_tag.get_text(separator=' ', strip=True) if status_tag else "N/A"  # Add space between tags
    
    # Ensure proper word spacing for the status
    status = add_spaces(status)
    
    # Extract the correct detailed summary, which is often within a <div> with id 'bill-summary'
    detailed_summary = "N/A"
    summary_section = bill_soup.find('div', id='bill-summary')

    if summary_section:
        # Join paragraphs with a space to avoid merging words
        all_paragraphs = summary_section.find_all('p')
        if all_paragraphs:
            detailed_summary = ' '.join([para.get_text(separator=' ', strip=True) for para in all_paragraphs])
    
    # Ensure proper word spacing for the detailed summary
    detailed_summary = add_spaces(detailed_summary)

    # Store the scraped data in a dictionary
    bills_data.append({
        'Title': bill_title,
        'Additional Info': status,
        'Detailed Summary': detailed_summary,
        'Link': f"https://www.congress.gov{link}"
    })

# Step 4: Convert the scraped data into a DataFrame
df = pd.DataFrame(bills_data)

# Close the WebDriver
#driver.quit()

df.tail()

,Title,Additional Info,Detailed Summary,Link
10,S.2678 - Outbound Investment Transparency Act of 2023118th Congress (2023-2024),"Sponsor: Sen. Cornyn, John [R-TX] (Introduced 07/27/2023) Committees: Senate - Banking, Housing, and Urban Affairs Latest Action: Senate - 07/27/2023 Read twice and referred to the Committee on Banking, Housing, and Urban Affairs. ( All Actions ) Tracker: Tip This bill has the status Introduced Here are the steps for Status of Legislation: Introduced Array\n(\n [action Date] => 2023-07-27\n [display Text] => Introduced in Senate\n [external Action Code] => 10000\n [description] => Introduced\n [chamber Of Action] => Senate\n)","There is one summary for S.2678. Bill summaries are authored by CRS . Outbound Investment Transparency Act of 2023 This bill requires U.S. persons to notify the Department of the Treasury when making investments in or engaging in other economic activity associated with North Korea, China, Russia, or Iran. This requirement applies to U.S. persons in certain sectors, including advanced semiconductors and microelectronics, artificial intelligence , and satellite-based communications. Covered activities that trigger the notification requirement include investing in an entity, establishing a venture or subsidiary, or acquiring certain positions in an entity in these countries. U.S. persons must notify Treasury 14 days before engaging in a covered activity. However, if the covered activity is a secured transaction, notification must be provided not later than 14 days after the transaction. Specific information or documentary material provided to Treasury is not subject to public disclosure, with certain exceptions. Treasury must at least annually report to Congress a summary of the notifications received under this bill. The Department of State must engage with allied countries to coordinate protocols, procedures, and information sharing regarding these investments. The Department of Justice has the authority to enforce this bill.",https://www.congress.gov/bill/118th-congress/senate-bill/2678?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=11
11,S.4295 - Financial Data Transparency Act of 2022117th Congress (2021-2022),"Sponsor: Sen. Warner, Mark R. [D-VA] (Introduced 05/24/2022) Committees: Senate - Banking, Housing, and Urban Affairs Latest Action: Senate - 05/24/2022 Read twice and referred to the Committee on Banking, Housing, and Urban Affairs. ( All Actions ) Tracker: Tip This bill has the status Introduced Here are the steps for Status of Legislation: Introduced Array\n(\n [action Date] => 2022-05-24\n [display Text] => Introduced in Senate\n [external Action Code] => 10000\n [description] => Introduced\n [chamber Of Action] => Senate\n)","There is one summary for S.4295. Bill summaries are authored by CRS . Financial Data Transparency Act of 2022 This bill requires federal financial regulatory agencies to adopt specified data standards with respect to format, searchability, and transparency. The bill also decreases, beginning September 30, 2031, the cap on the surplus funds of the Federal Reserve banks. (Amounts exceeding this cap are deposited in the general fund of the Treasury.) The Government Accountability Office must report on the feasibility of applying the taxonomy established by this bill to the wider federal government.",https://www.congress.gov/bill/117th-congress/senate-bill/4295?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=12
12,N/A,N/A,N/A,https://www.congress.gov/bill/117th-congress/house-bill/2989?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=13
13,H.R.4476 - Financial Transparency Act of 2019116th Congress (2019-2020),"Sponsor: Rep. Maloney, Carolyn B. [D-NY-12] (Introduced 09/24/2019) Committees: House - Financial Services; Agriculture Latest Action: House - 10/11/2019 Referred to the Subcommittee on Commodity Exchanges, Energy, and Credit. ( All Actions ) Tracker: Tip This bill has the status Introduced Here are the st

In [9]:
#extract individual columns from additional info column
def extract_sponsor(row):
    additional_info = row['Additional Info']
    
    # Regular expression to find all text after "Sponsor:" and up to "]"
    match = re.search(r'Sponsor:\s*([^\]]+\])', additional_info)
    
    if match:
        # Return the text after "Sponsor:" up to "]"
        return match.group(1).strip()
    else:
        # If "Sponsor" is not found, return None or an appropriate placeholder
        return "N/A"

# Apply the function to the DataFrame and create a new column "Sponsor"
df['Sponsor'] = df.apply(extract_sponsor, axis=1)


def extract_committee_info(row):
    additional_info = row['Additional Info']
    
    # Regular expression to find the text following "Committees" and between colons
    match = re.search(r'Committees:\s*([^:]+)', additional_info)
    
    if match:
        # Return the text that follows "Committees" until the next colon
        return match.group(1).strip()
    else:
        # If "Committees" is not found, return None or an appropriate placeholder
        return "N/A"

# Apply the function to the DataFrame and create a new column "Committee"
df['Committee'] = df.apply(extract_committee_info, axis=1)

def extract_introduced_date(row):
    additional_info = row['Additional Info']
    
    # Regular expression to find the text following "(Introduced" and before the closing parenthesis ")"
    match = re.search(r'\(Introduced\s+([^)]+)\)', additional_info)
    
    if match:
        # Return the date inside the parentheses after "(Introduced"
        return match.group(1).strip()
    else:
        # If "(Introduced" is not found, return None or an appropriate placeholder
        return "N/A"

# Apply the function to the DataFrame and create a new column "Introduced date"
df['Introduced date'] = df.apply(extract_introduced_date, axis=1)


def extract_latest_action(row):
    additional_info = row['Additional Info']
    
    # Regular expression to find the text following "Latest Action" from ":" to "."
    match = re.search(r'Latest Action:\s*([^\.]+)\.', additional_info)
    
    if match:
        # Return the text after "Latest Action" up to the period
        return match.group(1).strip()
    else:
        # If "Latest Action" is not found, return None or an appropriate placeholder
        return "N/A"

# Apply the function to the DataFrame and create a new column "latest action"
df['Latest Action'] = df.apply(extract_latest_action, axis=1)

df['Website']=df['Link']

df = df.drop(columns=['Additional Info','Link'])

driver.quit()

df

,Title,Detailed Summary,Sponsor,Committee,Introduced date,Latest Action,Website
0,H.R.2152 - AI PLAN Act119th Congress (2025-2026) |Get alerts,N/A,"Rep. Nunn, Zachary [R-IA-3]",House - Financial Services Latest Action,03/14/2025,House - 03/14/2025 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/119th-congress/house-bill/2152?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=1
1,H.R.1734 - Preventing Deep Fake Scams Act119th Congress (2025-2026) |Get alerts,N/A,"Rep. Pettersen, Brittany [D-CO-7]",House - Financial Services Latest Action,02/27/2025,House - 02/27/2025 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/119th-congress/house-bill/1734?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=2
2,H.Res.1600 - Expressing the sense of the House of Representatives with respect to the use ofartificialintelligencein the financial services and housing industries.118th Congress (2023-2024),N/A,"Rep. Mc Henry, Patrick T. [R-NC-10]",House - Financial Services Latest Action,11/26/2024,House - 11/26/2024 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/118th-congress/house-resolution/1600?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=3
3,H.R.9309 - Unleashing AI Innovation in Financial Services Act118th Congress (2023-2024),N/A,"Rep. Hill, J. French [R-AR-2]",House - Financial Services Latest Action,08/06/2024,House - 08/06/2024 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/118th-congress/house-bill/9309?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=4
4,S.4951 - Unleashing AI Innovation in Financial Services Act118th Congress (2023-2024),N/A,"Sen. Rounds, Mike [R-SD]","Senate - Banking, Housing, and Urban Affairs Latest Action",08/01/2024,"Senate - 08/01/2024 Read twice and referred to the Committee on Banking, Housing, and Urban Affairs",https://www.congress.gov/bill/118th-congress/senate-bill/4951?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=5
5,S.4870 - A bill to require reports onartificialintelligenceregulation in the financial services industry.118th Congress (2023-2024),N/A,"Sen. Rounds, Mike [R-SD]","Senate - Banking, Housing, and Urban Affairs Latest Action",07/30/2024,"Senate - 07/30/2024 Read twice and referred to the Committee on Banking, Housing, and Urban Affairs",https://www.congress.gov/bill/118th-congress/senate-bill/4870?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=6
6,H.R.7781 - AI PLAN Act118th Congress (2023-2024),"There is one summary for H.R.7781. Bill summaries are authored by CRS . Artificial Intelligence Practices, Logistics, Actions, and Necessities Act or the AI PLAN Act This bill requires the Department of the Treasury, the Department of Homeland Security (DHS), and the Department of Commerce (DOC) to submit a report that discusses the national and economic security risks posed by the use of artificial intelligence in the commission of financial crimes and a list of resources needed by federal agencies to combat these risks. The report must consider the risks of deepfakes, voice cloning, foreign election interference, synthetic identities, false flags, and general digital fraud. After submission of this report, Treasury, DHS, and DOC must make recommendations for legislation and best practices for businesses and government entities to address these risks.","Rep. Nunn, Zachary [R-IA-3]",House - Financial Services Committee Meetings,03/21/2024,House - 03/21/2024 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/118th-congress/house-bill/7781?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=7
7,S.3735 - Protecting Innovation in Investment Act118th Congress (2023-2024),"There is one summary for S.3735. Bill summaries are authored by CRS . Protecting Innovation in Investment Act This bill prohibits the Securities and Exchange Commission from finalizing the pr

In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL to scrape
fed_url = "https://www.federalreserve.gov/apps/proposals/search?searchTerm=artificial%20intelligence"
driver.get(fed_url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)  # Explicit wait (max 10 seconds)

# Initialize an empty list to store results
proposals_data = []

try:
    # Wait for elements to load
    title_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="reconnect-modal-container"]/div[2]/div[2]/div/div/div[3]/div[1]/h5/a')))
    summary_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="reconnect-modal-container"]/div[2]/div[2]/div/div/div[3]/div[3]/p')))
    latest_action_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="reconnect-modal-container"]/div[2]/div[2]/div/div/div[3]/div[4]/span')))

    # Extract text
    title = title_element.text.strip() if title_element else "N/A"
    detailed_summary = summary_element.text.strip() if summary_element else "N/A"
    latest_action = latest_action_element.text.strip() if latest_action_element else "N/A"

    # Extract the website link
    website_url = fed_url  # Store the source URL

    # Debugging output
    print(f"📌 Scraped Data:\nTitle: {title}\nDetailed Summary: {detailed_summary}\nLatest Action: {latest_action}\nWebsite: {website_url}")

    # Append data to the list
    proposals_data.append({
        "Title": title,
        "Detailed Summary": detailed_summary,
        "Latest Action": latest_action,
        "Website": website_url  # Store the website source
    })

    # Close the WebDriver
    driver.quit()

    # Convert list to DataFrame
    df2 = pd.DataFrame(proposals_data)

except Exception as e:
    print(f"🚨 Error occurred: {e}")
    driver.quit()


📌 Scraped Data:
Title: Request for Information and Comment on Financial Institutions' Use of Artificial Intelligence, including Machine Learning [OP-1743]
Detailed Summary: The agencies are gathering information and comments on financial institutions' use of artificial intelligence (AI), including machine learning (ML). The purpose of this request for information (RFI) is to understand respondents' views on the use of AI by financial institutions in their provision of services to customers and for other business or operational purposes; appropriate governance, risk management, and controls over AI; and any challenges in developing, adopting, and managing AI. The RFI also solicits respondents' views on the use of AI in financial services to assist in determining whether any clarifications from the agencies would be helpful for financial institutions' use of AI in a safe and sound manner and in compliance with applicable laws and regulations, including those related to consumer protectio

In [11]:
df = pd.concat([df, df2], ignore_index=True)
df

,Title,Detailed Summary,Sponsor,Committee,Introduced date,Latest Action,Website
0,H.R.2152 - AI PLAN Act119th Congress (2025-2026) |Get alerts,N/A,"Rep. Nunn, Zachary [R-IA-3]",House - Financial Services Latest Action,03/14/2025,House - 03/14/2025 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/119th-congress/house-bill/2152?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=1
1,H.R.1734 - Preventing Deep Fake Scams Act119th Congress (2025-2026) |Get alerts,N/A,"Rep. Pettersen, Brittany [D-CO-7]",House - Financial Services Latest Action,02/27/2025,House - 02/27/2025 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/119th-congress/house-bill/1734?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=2
2,H.Res.1600 - Expressing the sense of the House of Representatives with respect to the use ofartificialintelligencein the financial services and housing industries.118th Congress (2023-2024),N/A,"Rep. Mc Henry, Patrick T. [R-NC-10]",House - Financial Services Latest Action,11/26/2024,House - 11/26/2024 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/118th-congress/house-resolution/1600?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=3
3,H.R.9309 - Unleashing AI Innovation in Financial Services Act118th Congress (2023-2024),N/A,"Rep. Hill, J. French [R-AR-2]",House - Financial Services Latest Action,08/06/2024,House - 08/06/2024 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/118th-congress/house-bill/9309?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=4
4,S.4951 - Unleashing AI Innovation in Financial Services Act118th Congress (2023-2024),N/A,"Sen. Rounds, Mike [R-SD]","Senate - Banking, Housing, and Urban Affairs Latest Action",08/01/2024,"Senate - 08/01/2024 Read twice and referred to the Committee on Banking, Housing, and Urban Affairs",https://www.congress.gov/bill/118th-congress/senate-bill/4951?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=5
5,S.4870 - A bill to require reports onartificialintelligenceregulation in the financial services industry.118th Congress (2023-2024),N/A,"Sen. Rounds, Mike [R-SD]","Senate - Banking, Housing, and Urban Affairs Latest Action",07/30/2024,"Senate - 07/30/2024 Read twice and referred to the Committee on Banking, Housing, and Urban Affairs",https://www.congress.gov/bill/118th-congress/senate-bill/4870?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=6
6,H.R.7781 - AI PLAN Act118th Congress (2023-2024),"There is one summary for H.R.7781. Bill summaries are authored by CRS . Artificial Intelligence Practices, Logistics, Actions, and Necessities Act or the AI PLAN Act This bill requires the Department of the Treasury, the Department of Homeland Security (DHS), and the Department of Commerce (DOC) to submit a report that discusses the national and economic security risks posed by the use of artificial intelligence in the commission of financial crimes and a list of resources needed by federal agencies to combat these risks. The report must consider the risks of deepfakes, voice cloning, foreign election interference, synthetic identities, false flags, and general digital fraud. After submission of this report, Treasury, DHS, and DOC must make recommendations for legislation and best practices for businesses and government entities to address these risks.","Rep. Nunn, Zachary [R-IA-3]",House - Financial Services Committee Meetings,03/21/2024,House - 03/21/2024 Referred to the House Committee on Financial Services,https://www.congress.gov/bill/118th-congress/house-bill/7781?q=%7B%22search%22%3A%22Artificial+Intelligence%22%7D&s=1&r=7
7,S.3735 - Protecting Innovation in Investment Act118th Congress (2023-2024),"There is one summary for S.3735. Bill summaries are authored by CRS . Protecting Innovation in Investment Act This bill prohibits the Securities and Exchange Commission from finalizing the pr

In [12]:
df.to_excel(r'C:\Users\brian\OneDrive\Documents\Data Analytics\Personal Projects\Web Scrapper AI Regs\CongressRegsonAI.xlsx')